# Imports

In [ ]:
from pathlib import Path
import pandas as pd
from asapdiscovery.docking.analysis import calculate_rmsd_openeye
from asapdiscovery.data.openeye import load_openeye_sdf

In [ ]:
local_analysis = Path("/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/")

In [ ]:
csvs = local_analysis.glob("20231016_self_docked*/run_docking_oe.*-results.csv")

In [ ]:
df = pd.concat(map(pd.read_csv, csvs), ignore_index= True)

In [ ]:
df.groupby(["POSIT_method"]).nunique()

In [ ]:
df.columns

# Cleanup

In [ ]:
df.columns = ["Compound_ID", 
                 "Structure_Source", 
                 "Docked_File", 
                 "Pose_ID", 
                 "RMSD", 
                 "POSIT", 
                 "POSIT_Method", 
                 "Chemgauss4", 
                 "Clash", 
                 "SMILES", 
                 "GAT_Score",
                "SCHNET_score"]

In [ ]:
df["POSIT_R"] = 1-df["POSIT"]
df["Complex_ID"] = df.Compound_ID.apply(str) + "_" + df.Structure_Source

## remove non-p series structures

In [ ]:
p_only = df[df.Structure_Source.str.contains("Mpro-P")]

In [ ]:
p_only.groupby(["POSIT_Method"]).count()

## get intersection

In [ ]:
p_only.groupby(["Compound_ID", "POSIT_Method"]).count()

In [ ]:
p_only[p_only.Compound_ID == "ALP-POS-133e7cd9-2"]

In [ ]:
print([file for file in p_only[p_only.Compound_ID == "ALP-POS-133e7cd9-2"]["Docked_File"]])

# get intersection

In [ ]:
cmpds = p_only[p_only.POSIT_Method == "SHAPEFIT"]["Compound_ID"]

In [ ]:
intersection = p_only[p_only.Compound_ID.isin(cmpds)]

In [ ]:
intersection.groupby(["POSIT_Method"]).count()

In [ ]:
#cmpds = intersection[intersection.POSIT_Method == "FRED"]["Compound_ID"]

In [ ]:
intersection = intersection[intersection.Compound_ID.isin(cmpds)]

In [ ]:
intersection.groupby(["POSIT_Method"]).count()

In [ ]:
cmpds = intersection[intersection.POSIT_Method == "HYBRID"]["Compound_ID"]

In [ ]:
intersection = intersection[intersection.Compound_ID.isin(cmpds)]

In [ ]:
intersection.groupby(["POSIT_Method"]).count()

# save csv

In [ ]:
intersection.to_csv(local_analysis / "20231016_combined.csv")

In [ ]:
intersection

# Calculate RMSD

In [ ]:
docked_mols = [load_openeye_sdf(str(local_analysis.joinpath(*Path(file).parts[-3:]))) for file in intersection.Docked_File]

In [ ]:
ref_mols = [load_openeye_sdf(str(local_analysis.joinpath("sdf_lsf_array_p_only_by_name", f"{compound_id.strip()}.sdf")))  for compound_id in intersection.Compound_ID]

In [ ]:
len(docked_mols)

In [ ]:
len(ref_mols)

In [ ]:
rmsds = [calculate_rmsd_openeye(docked, ref) for docked, ref in zip(docked_mols, ref_mols)]

In [ ]:
intersection.RMSD = rmsds

In [ ]:
intersection.to_csv(local_analysis / "20231016_combined.csv")